In [97]:
import lightgbm as lgb
import lightgbm as lgb
## 導入Scikit-Learn的評量套件
from sklearn import metrics
from sklearn.metrics import mean_squared_error
## 導入Scikit-Learn的內建數據集
from sklearn. datasets import load_iris
## 導入Scikit-Learn用來拆分訓練集和測試集的套件
from sklearn.model_selection import train_test_split

In [98]:
import pandas as pd 
data_train = pd.read_excel("train.xlsx")

In [99]:
# Appendix: Data preprocessing (程式供參考用)
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler, StandardScaler  #建構standardization的物件

#name_list_outlier = ['手機螢幕溫度(平均)','手機螢幕溫度(最大)','手機背蓋溫度(平均)','平均APP運作指標/小時','充電時間指標',
                #'手機電池電量','手機外殼/包膜指標','外型平均曲率','上網頻率指標',
                #'通話頻率指標','新機/二手/老舊','內容容量比率','CPU效能等級']

name_list_outlier =['手機螢幕溫度(平均)','手機螢幕溫度(最大)', '手機背蓋溫度(平均)','平均APP運作指標/小時', '充電時間指標', '手機電池電量', '上網頻率指標', '通話頻率指標', '內容容量比率']
name_pre_outlier = ['Mobile Screen Temperature (Max)','Mobile Phone Back Cover Temperature (Average)','Average APP Operation Index/Hour','Mobile Phone Battery Level','Content Capacity Ratio']


#離群值處理
def  outlier_del(data_frame,name_list_outlier,n1=5,n2=5):
    for i, name in enumerate(name_list_outlier):
        print(name)
        print("Shape Of The Before Ouliers: ", data_frame[name].shape)
    
        # 计算IQR
        Q1 = np.percentile(data_frame[name], 25)
        Q3 = np.percentile(data_frame[name], 75)
        IQR = Q3 - Q1

        # 定义离群值范围
        lower_bound = Q1 - n1 * IQR
        upper_bound = Q3 + n2 * IQR

        # 过滤离群值
        filtered_entries = ((data_frame[name] >= lower_bound) & (data_frame[name] <= upper_bound))
        data_frame = data_frame[filtered_entries]
    
        print("Shape Of The After Ouliers: ",data_frame[name].shape)
        print('===================================================================================================================')
    return data_frame

#data_train=outlier_del(data_train,['手機螢幕溫度(平均)'],n1=0.1,n2=5)
#data_train=outlier_del(data_train,['手機背蓋溫度(平均)'],n1=0.1,n2=5)
#data_train=outlier_del(data_train,['上網頻率指標'],n1=0.1,n2=5)
data_train=outlier_del(data_train,name_list_outlier,n1=3,n2=3)

data_train.describe()

手機螢幕溫度(平均)
Shape Of The Before Ouliers:  (3903,)
Shape Of The After Ouliers:  (3903,)
手機螢幕溫度(最大)
Shape Of The Before Ouliers:  (3903,)
Shape Of The After Ouliers:  (3903,)
手機背蓋溫度(平均)
Shape Of The Before Ouliers:  (3903,)
Shape Of The After Ouliers:  (3900,)
平均APP運作指標/小時
Shape Of The Before Ouliers:  (3900,)
Shape Of The After Ouliers:  (3772,)
充電時間指標
Shape Of The Before Ouliers:  (3772,)
Shape Of The After Ouliers:  (3772,)
手機電池電量
Shape Of The Before Ouliers:  (3772,)
Shape Of The After Ouliers:  (3772,)
上網頻率指標
Shape Of The Before Ouliers:  (3772,)
Shape Of The After Ouliers:  (3772,)
通話頻率指標
Shape Of The Before Ouliers:  (3772,)
Shape Of The After Ouliers:  (3772,)
內容容量比率
Shape Of The Before Ouliers:  (3772,)
Shape Of The After Ouliers:  (3706,)


,手機螢幕溫度(平均),手機螢幕溫度(最大),手機背蓋溫度(平均),平均APP運作指標/小時,充電時間指標,手機電池電量,CPU跑分階級,手機外殼/包膜指標,外型平均曲率,上網頻率指標,通話頻率指標,新機/二手/老舊,內容容量比率,CPU效能等級,Underclocking
count,3706.000000,3706.000000,3706.000000,3706.000000,3706.000000,3706.000000,3706.000000,3706.000000,3706.000000,3706.000000,3706.000000,3706.000000,3706.000000,3706.000000,3706.000000
mean,38.912033,45.306485,52.010520,572.244075,0.225938,75.143304,3.284943,0.110056,0.014911,2442.457590,2096.191731,0.310038,0.870361,1.284943,0.430113
std,14.517281,15.514809,13.936551,495.392846,0.232016,14.021635,0.661063,0.092076,0.124063,1729.393433,1707.252233,0.581024,0.158460,0.661063,0.495159
min,8.025054,8.261905,2.562656,2.806544,0.000000,47.889610,3.000000,0.003690,-1.043000,0.675070,0.000000,0.000000,0.228778,1.000000,0.000000
25%,28.533183,34.738095,43.625566,223.700000,0.029412,62.788600,3.000000,0.058697,-0.055830,930.211806,546.550000,0.000000,0.809796,1.000000,0.000000
50%,38.738095,46.928571,53.186667,404.009705,0.145833,76.515176,3.000000,0.085691,0.005701,2191.000000,1816.342260,0.000000,0.938220,1.000000,0.000000
75%,50.047619,57.357143,60.820000,769.202491,0.371429,86.536305,3.000000,0.144959,0.078107,3730.500000,3338.000000,1.000000,0.986747,1.000000,1.000000
max,75.619048,79.547619,104.266667,2617.000000,0.966667,100.000000,6.000000,1.776212,1.208233,8410.000000,8129.000000,5.000000,1.000000,4.000000,1.000000


In [102]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler 
y_data = data_train['Underclocking'].to_numpy()
#feature_ex =['手機螢幕溫度(平均)', '手機背蓋溫度(平均)', '充電時間指標', '手機電池電量', 'CPU跑分階級', '上網頻率指標', '通話頻率指標', '內容容量比率']

feature_ex =['手機螢幕溫度(平均)','手機螢幕溫度(最大)','手機背蓋溫度(平均)','平均APP運作指標/小時','充電時間指標',
                '手機電池電量','手機外殼/包膜指標','外型平均曲率','上網頻率指標',
             '通話頻率指標','新機/二手/老舊','內容容量比率','CPU效能等級']
x_data = data_train[feature_ex].to_numpy()


def data_normalized(data):
    min_max_scaler = MinMaxScaler()
    data_normalized = min_max_scaler.fit_transform(data)
    return data_normalized
def data_standardized(data):
    standard_scaler = StandardScaler()
    data_standardized = standard_scaler.fit_transform(data)
    return data_standardized 
x_data=data_normalized(x_data)
x_data=data_standardized(x_data)
print(x_data) 
print(y_data) 

[[ 1.36915193  1.19430079 -0.63131647 ... -0.53367786  0.81345435
  -0.4310964 ]
 [ 0.92627017  0.80905615  0.43365308 ... -0.53367786  0.50449885
   1.08182335]
 [-0.56748124 -0.55388106 -1.10149668 ... -0.53367786  0.81822449
  -0.4310964 ]
 ...
 [ 2.18766303  1.89418747 -1.09586518 ... -0.53367786  0.57864234
  -0.4310964 ]
 [-1.36588898 -1.60217624 -0.74470272 ...  1.18765386  0.12126381
  -0.4310964 ]
 [ 0.11267997  0.11377399  0.42599831 ... -0.53367786 -0.89034652
  -0.4310964 ]]
[0 1 0 ... 0 0 0]


In [103]:
from sklearn.model_selection import train_test_split
#test_size=0.3 為比例-> training:test=7:3
trainX, testX, trainY, testY = train_test_split(x_data, y_data, test_size=0.2, random_state=42)



In [104]:

#trainX, testX, trainY, testY 
lgb_train = lgb.Dataset(trainX, trainY)
lgb_eval = lgb.Dataset(testX, testY, reference=lgb_train)

In [107]:


# specify your configurations as a dict
params = {
  'task': 'train',
  ## 算法類型
  'boosting': 'rf',
  'num_trees': 10000,
  'num_leaves': 20,
  'max_depth': 5,
  'learning_rate': 0.001,
  ## 構建樹時的特徵選擇比例
  'feature_fraction': 0.8,
  'feature_fraction_seed': 8,
  "bagging_fraction":0.5,
  ## k 表示每k次迭代就進行bagging
  'bagging_freq':5,
  ## 如果數據集樣本分布不均衡，可以幫助明顯提高準確率
  'is_unbalance': True,
  'verbose':0,
  ## 目標函數
  'objective': 'binary',
  ## 度量指標
  'metric':'binary_logloss',
  # 度量輸出的頻率
  'metric_freq': 1,
}
print("Starting training...")
# train
gbm = lgb.train(
    params, lgb_train, num_boost_round=20, valid_sets=lgb_eval, callbacks=[lgb.early_stopping(stopping_rounds=5)]
)
print("Saving model...")
lgbm.save_model('save_model.txt')

Starting training...
Training until validation scores don't improve for 5 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Early stopping, best iteration is:
[8]	valid_0's binary_logloss: 0.618957
Saving model...


In [106]:
y_pred = lgbm.predict(testX, num_iteration = lgbm.best_iteration)
print(y_pred)

LightGBMError: The number of features in data (13) is not the same as it was in training data (8).
You can set ``predict_disable_shape_check=true`` to discard this error, but please be aware what you are doing.

In [96]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# 假设y_pred是模型输出的概率
# 将概率转换为类别标签，阈值设置为0.5
y_pred_class = (y_pred > 0.48).astype(int)

# 现在我们可以安全地计算分类指标
accuracy = accuracy_score(testY, y_pred_class)
precision = precision_score(testY, y_pred_class)
recall = recall_score(testY, y_pred_class)
f1 = f1_score(testY, y_pred_class)

# 打印结果
print(f'Accuracy of the model: {accuracy}')
print(f'Precision of the model: {precision}')
print(f'Recall of the model: {recall}')
print(f'F1 Score of the model: {f1}')


Accuracy of the model: 0.6185983827493261
Precision of the model: 0.6104651162790697
Recall of the model: 0.32710280373831774
F1 Score of the model: 0.42596348884381335
